## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [78]:
import pymorphy2
import nltk
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [58]:
lines = list(open('news_train.txt', 'r', encoding='utf-8'))

In [59]:
import random
random.shuffle(lines)

In [60]:
train = lines[:10000]
test = lines[10000:]

In [61]:
from collections import Counter

Counter([line.split('\t')[0] for line in lines[:15000]])

Counter({'economics': 2080,
         'life': 2033,
         'science': 2156,
         'media': 2111,
         'travel': 289,
         'culture': 2053,
         'sport': 2215,
         'forces': 1225,
         'business': 554,
         'style': 284})

In [62]:
dict_train = {'labels': [], 'headings': [], 'texts': []}
dict_test = {'labels': [], 'headings': [], 'texts': []}

for line in train:
    label, heading, text = line.split('\t')
    dict_train['labels'].append(label)
    dict_train['headings'].append(re.findall(r'\b\w+\b', heading.lower()))
    dict_train['texts'].append(re.findall(r'\b\w+\b', text.lower()))

for line in test:
    label, heading, text = line.split('\t')
    dict_test['labels'].append(label)
    dict_test['headings'].append(re.findall(r'\b\w+\b', heading.lower()))
    dict_test['texts'].append(re.findall(r'\b\w+\b', text.lower()))

In [63]:
morph = pymorphy2.MorphAnalyzer()

dict_train['texts'] = [[morph.parse(word)[0].normal_form for word in text] for text in dict_train['texts']]
dict_test['texts'] = [[morph.parse(word)[0].normal_form for word in text] for text in dict_test['texts']]

In [68]:
model = Word2Vec(sentences=dict_train['texts'], workers=4)

In [69]:
model.wv.most_similar(positive=['жизнь'])

[('болезнь', 0.7818340063095093),
 ('девочка', 0.7690442800521851),
 ('женщина', 0.7455222606658936),
 ('родитель', 0.7219225168228149),
 ('собака', 0.7218487858772278),
 ('ребёнок', 0.712665319442749),
 ('мальчик', 0.6760157346725464),
 ('хорошо', 0.6539921164512634),
 ('кровь', 0.6531692743301392),
 ('глаз', 0.6516709327697754)]

In [70]:
model.wv.most_similar(positive=['смерть'])

[('развод', 0.7706078290939331),
 ('гибель', 0.735265851020813),
 ('вскоре', 0.7341554164886475),
 ('убийство', 0.7227871417999268),
 ('нападение', 0.7217468619346619),
 ('брак', 0.7181789875030518),
 ('девочка', 0.7101303339004517),
 ('задержание', 0.6926693320274353),
 ('погибший', 0.6722390651702881),
 ('мальчик', 0.6630831956863403)]

In [71]:
X_train = [' '.join(text) for text in dict_train['texts']]
y_train = dict_train['labels']
X_test = [' '.join(text) for text in dict_test['texts']]
y_test = dict_test['labels']

In [73]:
tfidf = TfidfVectorizer()

X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [79]:
RandCV_logreg = RandomizedSearchCV(LogisticRegression(), param_distributions={'C': np.arange(0.01, 1.01, 0.01)})
RandCV_naivebayes = RandomizedSearchCV(MultinomialNB(), param_distributions={'alpha': np.arange(0.5, 1.51, 0.01)})

RandCV_logreg.fit(X_train_vec, y_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'...
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0

In [80]:
y_pred_lreg = RandCV_logreg.best_estimator_.predict(X_test_vec)
print('Precision: ', precision_score(y_pred_lreg, y_test, average='weighted'))
print('Recall: ', recall_score(y_pred_lreg, y_test, average='weighted'))
print('F1 score: ', f1_score(y_pred_lreg, y_test, average='weighted'))
print('Accuracy: ', accuracy_score(y_pred_lreg, y_test))


Precision:  0.874984534463844
Recall:  0.8532
F1 score:  0.8602019571375445
Accuracy:  0.8532


In [81]:
RandCV_naivebayes.fit(X_train_vec, y_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=MultinomialNB(alpha=1.0, class_prior=None,
                                           fit_prior=True),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'alpha': array([0.5 , 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 ,
       0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71,
       0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82,
       0.83...
       0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.  , 1.01, 1.02, 1.03, 1.04,
       1.05, 1.06, 1.07, 1.08, 1.09, 1.1 , 1.11, 1.12, 1.13, 1.14, 1.15,
       1.16, 1.17, 1.18, 1.19, 1.2 , 1.21, 1.22, 1.23, 1.24, 1.25, 1.26,
       1.27, 1.28, 1.29, 1.3 , 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37,
       1.38, 1.39, 1.4 , 1.41, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48,
       1.49, 1.5 ])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                 

In [82]:
y_pred_nb = RandCV_naivebayes.predict(X_test_vec)
print('Precision: ', precision_score(y_pred_nb, y_test, average='weighted'))
print('Recall: ', recall_score(y_pred_nb, y_test, average='weighted'))
print('F1 score: ', f1_score(y_pred_nb, y_test, average='weighted'))
print('Accuracy: ', accuracy_score(y_pred_nb, y_test))

Precision:  0.8783956455637224
Recall:  0.8
F1 score:  0.8309893778025075
Accuracy:  0.8
